In [90]:
from api_connection import NotionAPIConnector

connector = NotionAPIConnector()
raw_response:dict = connector.main()

In [97]:
class ResultFetcher:
    
    PAGE_COL_NAME = "Pages"
    # DESC_COL_NAME = "Description"
    
    """ 
    Fetch the results of the raw json response 
    """
    
    def __init__(self, raw_response:dict):
        self.raw_response = raw_response
        
    
    @staticmethod
    def create_initial_dict(raw_response:dict) -> dict:
        """ 
        Return a dict with the main subject pages as key and the page id as title
        """
        # On itére dans la database directement là (première couche)
        results = raw_response["results"]
        
        initial_dict = {}
        
        page:dict
        for page in results:
            page_id = page['id']
            page_name = page['properties']['Pages']['title'][0]['plain_text']
            initial_dict[page_name] = page_id
            
        return initial_dict
    
    
""" 
Maintenant qu'on a les pages principales il faut ; 
- Qu'on itère dans chaque main page 
- Qu'on détecte les pages enfants
- Qu'on récupére les subpages des pages enfants 
- Si jamais une subpage ne contient que des pages et pas de texte, il ne faut pas l'ajouter dans le système 
- Il faut garder le tag (le nom principale qui nous vient du premier dict) de chaque sous page

"""
    
        
fetcher = ResultFetcher(raw_response=raw_response)

fetcher.create_initial_dict(raw_response)

fcbca7da-3517-4163-9362-21525d5784f1 : Ton pere 
80497c93-1232-4710-b14d-b45a27563932 : ta mere
e54dd41c-3ac2-47bb-b721-8cbb744990de : Page test


In [92]:
len(raw_response["results"])

3

In [93]:
page_to_fetch = json_response["results"][0]
block_id = get_page_id(page_to_fetch)
block_id

'31e3260b-f9a4-4b7c-bfc9-577af46712a3'

In [94]:
# Get subpages
url = f'https://api.notion.com/v1/blocks/{block_id}/children'


response = requests.get(url, headers=headers)
print(len(response.json()["results"])) # 2 subpages

KeyError: 'results'

In [ ]:
# id de la subpage, le but c'est de récup son URL
subpage_id = response.json()["results"][0]["id"]

url = f'https://api.notion.com/v1/blocks/{subpage_id}/children'

response = requests.get(url, headers=headers)

IndexError: list index out of range

In [ ]:
url = f'https://api.notion.com/v1/pages/{subpage_id}'

response = requests.get(url, headers=headers)

response.json()

{'object': 'page',
 'id': 'e266eee4-b513-4c48-a793-7dfa05b5240f',
 'created_time': '2024-08-03T14:39:00.000Z',
 'last_edited_time': '2024-08-22T15:14:00.000Z',
 'created_by': {'object': 'user',
  'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
 'last_edited_by': {'object': 'user',
  'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
 'cover': None,
 'icon': None,
 'parent': {'type': 'page_id',
  'page_id': 'e54dd41c-3ac2-47bb-b721-8cbb744990de'},
 'archived': False,
 'in_trash': False,
 'properties': {'title': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text',
     'text': {'content': 'Propal Raphael ', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'Propal Raphael ',
     'href': None}]}},
 'url': 'https://www.notion.so/Propal-Raphael-e266eee4b5134c48a7937dfa05b5240f',
 'public_url': None,
 'request_id': '60c1ffe1-a1ac-4cd9-b5d

In [ ]:
response.json()

{'object': 'list',
 'results': [{'object': 'block',
   'id': 'e9921fe3-c3bd-4f0d-b507-20d47d03fc19',
   'parent': {'type': 'page_id',
    'page_id': 'e266eee4-b513-4c48-a793-7dfa05b5240f'},
   'created_time': '2024-08-22T15:11:00.000Z',
   'last_edited_time': '2024-08-22T15:11:00.000Z',
   'created_by': {'object': 'user',
    'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
   'last_edited_by': {'object': 'user',
    'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [], 'color': 'default'}},
  {'object': 'block',
   'id': '7c547acf-50de-431a-8b95-59858a152330',
   'parent': {'type': 'page_id',
    'page_id': 'e266eee4-b513-4c48-a793-7dfa05b5240f'},
   'created_time': '2024-08-22T15:11:00.000Z',
   'last_edited_time': '2024-08-22T15:11:00.000Z',
   'created_by': {'object': 'user',
    'id': '095667f4-633c-4a3c-9ae2-b0da56bd7e27'},
   'last_edited_by': {'object': 'user',